# Tensorflow Graph and GraphDef

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.core.framework import graph_pb2
from tensorflow.core.framework import node_def_pb2
from tensorflow.core.framework import types_pb2
from tensorflow.core.framework import attr_value_pb2


In [ ]:
tf.__version__

In [ ]:
empty_graph = tf.Graph() # setup a empty graph
empty_graph_def = graph_pb2.GraphDef()
empty_graph_def_from_empty_graph = empty_graph.as_graph_def()

Graph

In [ ]:
x = tf.placeholder(tf.int32, shape=[None, 5], name="x")
y = tf.placeholder(tf.int32, shape=[None, 5], name="y")
output = tf.add(x * y, 1, name="output")

In [ ]:
default_graph = tf.get_default_graph()
for op in default_graph.get_operations():
    print(op)

In [ ]:
with tf.Graph().as_default() as new_graph:
    x = tf.placeholder(tf.int32, shape=[None, 5], name="x")
    y = tf.placeholder(tf.int32, shape=[None, 5], name="y")
    output = x + y

for op in new_graph.get_operations():
    print(op)

In [ ]:
default_graph = tf.get_default_graph()
for op in default_graph.get_operations():
    print(op)

In [ ]:
with tf.Session() as sess:
    x = sess.graph.get_tensor_by_name("x:0")
    y = sess.graph.get_tensor_by_name("y:0")
    output = sess.graph.get_tensor_by_name("output:0")
    out_ndarray = sess.run(output, feed_dict = {x: np.ones([4, 5]), y: np.ones([4, 5])})
    print(out_ndarray)

In [ ]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        try: 
            x = sess.graph.get_tensor_by_name("x:0")
            y = sess.graph.get_tensor_by_name("y:0")
            output = sess.graph.get_tensor_by_name("output:0")
            out_ndarray = sess.run(output, feed_dict = {x: np.ones([4, 5]), y: np.ones([4, 5])})
            print(out_ndarray)
        except KeyError as e:
            print(e)


In [ ]:
tf.reset_default_graph()

GraphDef

In [ ]:
with tf.Graph().as_default() as tfgraph:
    x = tf.placeholder(tf.float32, shape=[None, 5], name="x")
    y = tf.constant(np.ones(shape=[5, 5]), name="y", dtype=tf.float32)
    output = tf.matmul(x, y, transpose_b=True)

In [ ]:
tfgraph_def = tfgraph.as_graph_def()

In [ ]:
with tf.Graph().as_default() as imported_graph:
    tf.import_graph_def(tfgraph_def, name="")

assert tfgraph_def == imported_graph.as_graph_def()

In [ ]:
for node in tfgraph_def.node:
    print(node)

In [ ]:
tfgraph_def_copy = graph_pb2.GraphDef()
tfgraph_def_copy.CopyFrom(tfgraph_def)

In [ ]:
new_node_add = tfgraph_def_copy.node.add()
new_node_add.op = "Mul"
new_node_add.name = "mul"
new_node_add.input.extend(["x", "y"])
new_node_add.attr["T"].CopyFrom(attr_value_pb2.AttrValue(type=types_pb2.DT_FLOAT))

# name: "mul"
# op: "Mul"
# input: "x"
# input: "y"
# attr {
#   key: "T"
#   value {
#     type: DT_INT32
#   }
# }

tfgraph_def_copy

In [ ]:
new_node = node_def_pb2.NodeDef()
new_node.op = "Identity"
new_node.name = "matmul"
new_node.input.append("MatMul")
new_node.attr["T"].CopyFrom(attr_value_pb2.AttrValue(type=types_pb2.DT_FLOAT))
print(new_node)
tfgraph_def_copy.node.append(new_node)

In [ ]:
with tf.Graph().as_default() as add_indentity:
    tf.import_graph_def(tfgraph_def_copy, name = "")
    print(add_indentity.get_operation_by_name("matmul"))

In [ ]:
temp_tfgraph_def = graph_pb2.GraphDef()
temp_tfgraph_def.CopyFrom(tfgraph_def_copy)

for node in temp_tfgraph_def.node:
    if node.name in ("matmul", "mul"):
        tfgraph_def_copy.node.remove(node)

assert tfgraph_def_copy == tfgraph_def

FP32 to FP16

In [ ]:
with tf.Graph().as_default() as constant_tensor:
    a = tf.constant(1, name = "const_a", dtype = tf.float32)
    b = tf.constant(1, name = "const_b", dtype = tf.float32)
    c = a * b

constant_tensor_graph_def = constant_tensor.as_graph_def()

In [ ]:
constant_tensor_graph_def

In [ ]:
# change const_a
constant_tensor_graph_def.node[0].attr["dtype"].type = types_pb2.DT_HALF
values = tf.make_ndarray(constant_tensor_graph_def.node[0].attr['value'].tensor).astype(np.float16)
constant_tensor_graph_def.node[0].attr['value'].tensor.CopyFrom(tf.make_tensor_proto(values, dtype=tf.float16))
# change const_b
constant_tensor_graph_def.node[1].attr["dtype"].type = types_pb2.DT_HALF
values = tf.make_ndarray(constant_tensor_graph_def.node[0].attr['value'].tensor).astype(np.float16)
constant_tensor_graph_def.node[1].attr['value'].tensor.CopyFrom(tf.make_tensor_proto(values, dtype=tf.float16))
# change mul
constant_tensor_graph_def.node[2].attr['T'].type = types_pb2.DT_HALF

In [ ]:
with tf.Graph().as_default():
    out = tf.import_graph_def(constant_tensor_graph_def, name = "", return_elements=["mul:0"])
    with tf.Session() as sess:
        out_ndarray, *_ = sess.run(out)
        print(out_ndarray.dtype)

Type

In [ ]:
def np_type_to_tf_type(np_type):
    return tf.as_dtype(np_type)

np_type_to_tf_type(np.int32)

In [ ]:
def tf_type_to_np_type(tf_type):
    return tf_type.as_numpy_dtype

tf_type_to_np_type(tf.int64)

In [ ]:
def tf_type_to_datatype_enum(tf_type):
    return tf_type.as_datatype_enum

tf_type_to_datatype_enum(tf.float32) == types_pb2.DT_FLOAT